일반화 성능도 있기 때문에 전체 데이터셋에 대해서 성능 평가를 해봐야 함

In [1]:
import pickle
with open('../models/BernoulliNBl30_r15.pkl', 'rb') as f:
    classifier = pickle.load(f)
    
from py.utils import load_data
head = 'l30_r15'
directory = '../data/'
x, y, x_words, vocabs = load_data(head, directory)

x shape = (15715, 4551)
y shape = (15715,)
# features = 4551
# L words = 15715


In [2]:
word_frequency = x.sum(axis=1)
word_frequency

matrix([[100],
        [295],
        [104],
        ..., 
        [ 31],
        [ 93],
        [199]])

In [7]:
def get_sample_l(ri, topk=5):
    nonzero = x[:,ri].nonzero()[0]
    base = min(50, len(nonzero)//2)
    return [x_words[l] for l in sorted(nonzero, key=lambda x:word_frequency[x,0], reverse=True)[base:base+topk]]

In [10]:
base = 0
topk = 100
for r, coef in sorted(enumerate(classifier.coef_[0]), key=lambda x:x[1], reverse=True)[base :base + topk]:
    print('%s - %s (%.3f)' % (get_sample_l(r), vocabs[r], coef))

['안', '지금', '이상', '모습', '사용'] - 의 (-0.184)
['어머니', '중', '이것', '여기', '사이'] - 에 (-0.261)
['무엇', '보이', '마음', '얼굴', '전'] - 도 (-0.595)
['지금', '이상', '맞', '모습', '사용'] - 을 (-0.620)
['선생', '사랑', '삶', '돈', '데'] - 이 (-0.647)
['지금', '이상', '맞', '모습', '사용'] - 은 (-0.737)
['안', '이상', '모습', '사용', '길'] - 에서 (-0.830)
['세상', '현실', '과정', '학생', '글'] - 과 (-0.847)
['변화', '글', '존재', '누구', '시'] - 를 (-0.941)
['시대', '문화', '친구', '어디', '개'] - 가 (-0.951)
['죽', '사이', '사', '맞', '나오'] - 는 (-1.020)
['이름', '여성', '한국', '세상', '과정'] - 으로 (-1.041)
['어디', '현실', '개', '변화', '글'] - 로 (-1.064)
['존재', '누구', '시', '저', '자체'] - 와 (-1.125)
['죽', '사이', '안', '지금', '이상'] - 이다 (-1.226)
['모습', '길', '다음', '세계', '자기'] - 인 (-1.291)
['내용', '책', '힘', '뿐', '뜻'] - 이나 (-1.305)
['몸', '선생', '자리', '사랑', '삶'] - 하고 (-1.369)
['몸', '자리', '사랑', '뒤', '삶'] - 에는 (-1.414)
['사이', '지금', '이상', '모습', '길'] - 만 (-1.462)
['다음', '세계', '자기', '방법', '나라'] - 까지 (-1.488)
['돈', '역사', '정부', '주장', '관계'] - 들이 (-1.504)
['발견', '현상', '대답', '인식', '평가'] - 한 (-1.550)
['관련', '노력', '

In [31]:
r = '치곤'.strip()
ri = vocabs.index(r)
for l in x[:,ri].nonzero()[0]:
    print('%s - %s' % (x_words[l], r))

대답 - 치곤
작품 - 치곤
사고 - 치곤
압구정동 - 치곤
현장 - 치곤
도둑 - 치곤
곤충 - 치곤
연세 - 치곤
내용 - 치곤
건망증 - 치곤
행실 - 치곤
발걸음 - 치곤
대가 - 치곤
아이 - 치곤
장사 - 치곤
이야기 - 치곤
농담 - 치곤
여자 - 치곤
얘기 - 치곤
